# Install


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

import pandas as pd
from dotenv import load_dotenv

import utils
import mock_data
import model

load_dotenv("../user.env")

True

# Process data from API (mocked)


In [3]:
def extract_transactions_and_balance_from_response(account_id: str, transactions_raw):
    def combine_pending_and_booked_transactions(transactions: list[dict]):
        booked_df = pd.json_normalize(transactions.get("booked"))
        if booked_df is not None:
            booked_df["status"] = "booked"
        pending_df = pd.json_normalize(transactions.get("pending"))
        if pending_df is not None:
            pending_df["status"] = "pending"

        df = pd.concat([booked_df, pending_df])
        transactions = df.to_dict(orient="records")
        return transactions

    def find_account_transactions():
        # loop through looking for account id
        result = None
        finished = False

        for acc in transactions_raw:
            acc_id = acc[0]
            if acc_id == account_id:
                result = acc
        finished = True

        if finished and result is None:
            Exception(f"could not find account_id {account_id} in account details data")

        return result

    def extract_transactions(result):
        acc_transactions = result[1][0].get("transactions")
        return acc_transactions

    def extract_balances(result):
        acc_balances = result[1][1].get("balances")
        return acc_balances

    def add_account_id_to_transactions(transactions: list[dict], account_id: str):
        result = []

        for transaction in transactions:
            updated_dict = transaction
            updated_dict["account_id"] = account_id
            result.append(updated_dict)

        return result

    # parse
    result = find_account_transactions()
    acc_transactions = extract_transactions(result)
    acc_balances = extract_balances(result)

    # post process
    acc_transactions = combine_pending_and_booked_transactions(acc_transactions)
    acc_transactions = add_account_id_to_transactions(
        transactions=acc_transactions, account_id=account_id
    )

    return acc_transactions, acc_balances


def populate_user_accounts_data(
    requisition: model.Requisition,
    transactions,
    account_details: model.UserAccountDetails,
):
    # Populate accounts
    accounts = [
        parse_account_dict(acc, id=requisition.accounts[idx])
        for idx, acc in enumerate(account_details)
    ]

    for account_id in requisition.accounts:
        # acc_position = requisition.get("accounts").index(account_id)
        acc_position = requisition.accounts.index(account_id)
        account_name = accounts[acc_position].get("account_name")
        print(
            f"account_position={acc_position}, account_name={account_name}, account_id={account_id}"
        )

        transactions, balances = extract_transactions_and_balance_from_response(
            account_id, transactions_raw=transactions_raw
        )

        accounts[acc_position]["transactions"] = transactions
        accounts[acc_position]["balances"] = balances

    return accounts

In [5]:
user_data, transactions, account_mapping = mock_data.get_mocked_data()

In [6]:
requisition = user_data.requisition
account_details = user_data.account_details

In [7]:
user_data.account_details

UserAccountDetails(credit_cards=[CreditCardDetails(resource_id='c4e9a964-b676-43c1-81f9-5479f8894604', currency='GBP', masked_pan='**4572', details='STAFF ALL IN ONE CREDIT CARD')], current_accounts=[CurrentAccountDetails(resource_id='cbc85e1c-cdbc-4e3a-9918-03b74d79e719', bban='09012921263671', currency='GBP', name='joint account', cash_account_type='CACC'), CurrentAccountDetails(resource_id='a2b51663-5cf3-43d6-866e-65e5c63542bc', bban='09012916688559', currency='GBP', name='single account', cash_account_type='CACC')])

In [ ]:
account_details.credit_cards

In [ ]:
requisition.accounts

In [ ]:
for account_id in requisition.accounts:
    # acc_position = requisition.get("accounts").index(account_id)
    acc_position = requisition.accounts.index(account_id)
    account_name = accounts[acc_position].get("account_name")
    print(
        f"account_position={acc_position}, account_name={account_name}, account_id={account_id}"
    )

    transactions, balances = extract_transactions_and_balance_from_response(
        account_id, transactions_raw=transactions_raw
    )

    accounts[acc_position]["transactions"] = transactions
    accounts[acc_position]["balances"] = balances

In [ ]:
user_accounts_data = populate_user_accounts_data(
    requisition=user_data.requisition,
    transactions=transactions,
    account_details=user_data.account_details,
)

In [ ]:
type(user_accounts_data)

# Parsed data to domain model

accounts


In [ ]:
def get_account_data(account_data):
    result_dict = {}
    balances: list[dict] = account_data.get("balances")
    data_list = utils.flatten_and_remove_duplicates_from_dictionary(balances)
    for entry in data_list:
        balance_type = entry.get("balanceType")
        amount = entry.get("amount")

        if balance_type and amount:
            key = f"{balance_type}Balance"
            result_dict[key] = amount

    result_dict["currency"] = data_list[0]["currency"]
    result_dict["referenceDate"] = data_list[0]["referenceDate"]
    result_dict = utils.clean_column_names(result_dict)

    acc_detail = {
        "account_id": account_data.get("account_id"),
        "account_name": account_data.get("account_name"),
        "account_type": account_data.get("account_type"),
        "currency": account_data.get("currency"),
    }
    acc_detail.update(result_dict)

    return acc_detail


# clean
def transactions_dict_to_model(account_data: dict, account_id: str):
    transactions = utils.clean_column_names(account_data.get("transactions"))
    transactions = [model.Transaction.from_dict(d) for d in transactions]
    filtered_transactions = [t for t in transactions if t.account_id == account_id]
    return transactions

In [ ]:
def get_user_selection(selected_account: int = 0):
    account_ids = [d.get("account_id") for d in user_accounts_data]
    if selected_account >= len(account_ids):
        raise ValueError("invalid entry")
    account_id = account_ids[selected_account]
    account_position = account_ids.index(account_id)
    return account_position


account_position = get_user_selection()

In [ ]:
# user selects account
user_accounts = []
for i in range(0, len(user_accounts_data)):
    try:
        account_data = user_accounts_data[i]
        account_type = account_data.get("account_type")
        # initialise account
        if account_type == "CACC":
            account = model.BankAccount.from_dict(get_account_data(account_data))
        elif account_type == "CARD":
            account = model.CreditCard.from_dict(get_account_data(account_data))

        # initialise transactions
        transactions = transactions_dict_to_model(account_data, account.account_id)
        account.add_transactions(transactions=transactions)
        user_accounts.append(account)
    except Exception as e:
        print(f"failed to process account {i}")
        raise e

In [ ]:
user_accounts

In [ ]:
user = model.User(user_name="Matthew")

In [ ]:
user = model.User(user_name="Matthew")
_ = [user.add_account(account) for account in user_accounts]

In [ ]:
transactions = user.get_transactions()
transactions_by_amount = user.get_transactions_by_amount(100, 200)
transactions_by_amount = user.get_transactions_by_amount(400, 500)

In [ ]:
from collections import OrderedDict


def find_common_transactions():
    ...


def find_common_dicts(*lists):
    def dict_to_ordered_tuple(d):
        return tuple(OrderedDict(sorted(d.items())).items())

    sets = [set(dict_to_ordered_tuple(d) for d in lst) for lst in lists]

    common_set = set.intersection(*sets)

    result = [dict(t) for t in common_set]
    return result

In [ ]:
queries = [user.get_transactions(), user.get_transactions_by_amount(-100, -10)]
dicts = [[d.to_dict() for d in query] for query in queries]
common_dicts = find_common_dicts(*dicts)

In [ ]:
query_response = [model.Transaction(**t) for t in common_dicts]

In [ ]:
def transactions_to_dataframe(transactions: list[model.Transaction]):
    return pd.DataFrame(query_response).dropna(axis=1)

In [ ]:
transactions_to_dataframe(query_response)